In [1]:
import pandas as pd
import cv2
import numpy as np
import keras
import matplotlib.pyplot as plt
import os

from glcm import *
from basics import *
# from classifier import *

from PIL import Image
from scipy.ndimage import generic_filter
from skimage.morphology import medial_axis
import networkx as nx
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings('ignore')

2022-12-02 20:41:22.985939: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 20:41:23.149680: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 20:41:23.186577: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-02 20:41:24.212117: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier as RF


def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print(f"Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Precision Score: {precision_score(y_train, pred, pos_label='drone') * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:")
        print(f"{clf_report}")
        print("_______________________________________________")
        disp = ConfusionMatrixDisplay(confusion_matrix(y_train, pred), display_labels = clf.classes_)
        disp.plot()
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print(f"Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"Precision Score: {precision_score(y_test, pred, pos_label='drone') * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:")
        print(f"{clf_report}")
        print("_______________________________________________")
        disp = ConfusionMatrixDisplay(confusion_matrix(y_test, pred), display_labels = clf.classes_)
        disp.plot()
        
def classify(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    print_score(model, X_train, y_train, X_test, y_test, train=False)
    print(f'Params: {model.get_params()}')

def gridsearch(model, param_grid, X_train, y_train, X_test, y_test, name):
    sc = StandardScaler()
    X_train_scale = sc.fit_transform(X_train)
    X_test_scale = sc.fit_transform(X_test)
    grid = GridSearchCV(model, param_grid, refit=True, verbose=1, cv=5)
    grid.fit(X_train_scale, y_train)
    best_params = grid.best_params_
    print(f"Best params: {best_params}")
    if name == 'SVC': 
        clf = SVC(**best_params)
    elif name == 'RF': 
        clf = RF(**best_params)
    clf.fit(X_train_scale, y_train)
    print_score(clf, X_train_scale, y_train, X_test_scale, y_test, train=False)

In [3]:
AUG_DRONE = 'Data/trajectories_augmented/drone/'
AUG_BIRD = 'Data/trajectories_augmented/bird/'

In [4]:
def get_coordinates(im):
    inverse = cv2.bitwise_not(im)

    gray = cv2.cvtColor(inverse, cv2.COLOR_BGR2GRAY);

    thinned = cv2.ximgproc.thinning(gray)

    points = np.argwhere(thinned == 255)
    x,y  = points[:,1], points[:,0]
    points = np.c_[x,y]
    clf = NearestNeighbors(n_neighbors=3).fit(points)
    G = clf.kneighbors_graph()
    T = nx.from_scipy_sparse_matrix(G)
    order = list(nx.dfs_preorder_nodes(T, 0))

    xx = x[order]
    yy = y[order]
    paths = [list(nx.dfs_preorder_nodes(T, i)) for i in range(len(points))]

    mindist = np.inf
    minidx = 0
    for i in range(len(points)):
        p = paths[i]           # order of nodes
        ordered = points[p]    # ordered nodes
        # find cost of that order by the sum of euclidean distances between points (i) and (i+1)
        cost = (((ordered[:-1] - ordered[1:])**2).sum(1)).sum()
        if cost < mindist:
            mindist = cost
            minidx = i

    opt_order = paths[minidx]
    xx = x[opt_order]
    yy = y[opt_order]
    
    res_img = cv2.bitwise_not(thinned)
    return xx, yy, res_img

In [5]:
colnames = ['file','frames','turn_angle',
            'y_freq','curvature','velocity','acceleration',
            'label']
final_df = pd.DataFrame(columns = colnames)
glcm_df = pd.DataFrame()
contrast_df = pd.DataFrame()
dissim_df = pd.DataFrame()
homogen_df = pd.DataFrame()
energy_df = pd.DataFrame()
corr_df = pd.DataFrame()
asm_df = pd.DataFrame()

In [6]:
def keep_every_nth(lst, n):
    return lst[::n]

In [7]:
for filename in os.listdir(AUG_DRONE):
    xcenter, ycenter, line = get_coordinates(cv2.imread(AUG_DRONE + filename))
    if len(xcenter) < 320 or len(ycenter) < 320:
        continue
    else:
        while len(xcenter)>320:
            xcenter = keep_every_nth(xcenter,3)
            ycenter = keep_every_nth(ycenter,3)
    img = plot_trajectory(xcenter, ycenter)
    file = 'drone_' + filename
    matrix_coocurrence = get_glcm_features(img)
    label = 'drone'
    final_df = final_df.append({'file': file,
                    'frames': len(xcenter),
                    'turn_angle': turn_angle(xcenter, ycenter),
#                         'x_freq': frequency(xcenter),
                    'y_freq': frequency(ycenter),
                    'curvature': curvature(xcenter, ycenter),
                    'velocity': velocity(np.column_stack((xcenter, ycenter))),
                    'acceleration': acceleration(np.column_stack((xcenter, ycenter))),
                    'label': label}, ignore_index=True)
    glcm_df = pd.concat([glcm_df, pd.DataFrame(matrix_coocurrence.flatten())], axis=1)
    contrast_df = pd.concat([contrast_df, pd.DataFrame(contrast_feature(matrix_coocurrence).flatten())], axis=1)
    dissim_df = pd.concat([dissim_df, pd.DataFrame(dissimilarity_feature(matrix_coocurrence).flatten())], axis=1)
    homogen_df = pd.concat([homogen_df, pd.DataFrame(homogeneity_feature(matrix_coocurrence).flatten())], axis=1)
    energy_df = pd.concat([energy_df, pd.DataFrame(energy_feature(matrix_coocurrence).flatten())], axis=1)
    corr_df = pd.concat([corr_df, pd.DataFrame(correlation_feature(matrix_coocurrence).flatten())], axis=1)
    asm_df = pd.concat([asm_df, pd.DataFrame(asm_feature(matrix_coocurrence).flatten())], axis=1)

/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be 

In [8]:
for filename in os.listdir(AUG_BIRD):
    xcenter, ycenter, line = get_coordinates(cv2.imread(AUG_BIRD + filename))
    if len(xcenter) < 320 or len(ycenter) < 320:
        continue
    else:
        while len(xcenter)>320:
            xcenter = keep_every_nth(xcenter,3)
            ycenter = keep_every_nth(ycenter,3)
    img = plot_trajectory(xcenter, ycenter)
    file = 'bird_' + filename
    matrix_coocurrence = get_glcm_features(img)
    label = 'bird'
    final_df = final_df.append({'file': file,
                    'frames': len(xcenter),
                    'turn_angle': turn_angle(xcenter, ycenter),
#                         'x_freq': frequency(xcenter),
                    'y_freq': frequency(ycenter),
                    'curvature': curvature(xcenter, ycenter),
                    'velocity': velocity(np.column_stack((xcenter, ycenter))),
                    'acceleration': acceleration(np.column_stack((xcenter, ycenter))),
                    'label': label}, ignore_index=True)
    glcm_df = pd.concat([glcm_df, pd.DataFrame(matrix_coocurrence.flatten())], axis=1)
    contrast_df = pd.concat([contrast_df, pd.DataFrame(contrast_feature(matrix_coocurrence).flatten())], axis=1)
    dissim_df = pd.concat([dissim_df, pd.DataFrame(dissimilarity_feature(matrix_coocurrence).flatten())], axis=1)
    homogen_df = pd.concat([homogen_df, pd.DataFrame(homogeneity_feature(matrix_coocurrence).flatten())], axis=1)
    energy_df = pd.concat([energy_df, pd.DataFrame(energy_feature(matrix_coocurrence).flatten())], axis=1)
    corr_df = pd.concat([corr_df, pd.DataFrame(correlation_feature(matrix_coocurrence).flatten())], axis=1)
    asm_df = pd.concat([asm_df, pd.DataFrame(asm_feature(matrix_coocurrence).flatten())], axis=1)

/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be 

In [9]:
final_df

,file,frames,turn_angle,y_freq,curvature,velocity,acceleration,label
0,drone_aug_18.png,163,0.648666,0.184049,0.230862,4.248385,0.028669,drone
1,drone_aug_166.png,320,-0.096697,0.093750,0.302611,4.826906,0.094006,drone
2,drone_aug_63.png,119,0.125782,0.252101,0.362415,13.684275,0.248851,drone
3,drone_aug_161.png,130,0.925548,0.230769,0.275335,3.330667,0.001340,drone
4,drone_aug_121.png,254,0.109802,0.118110,0.253963,4.576391,-0.007963,drone
...,...,...,...,...,...,...,...,...
148,bird_aug_56.png,127,0.370922,0.236220,0.394207,13.190120,0.295977,bird
149,bird_aug_1.png,111,-0.494136,0.270270,0.298891,3.775308,0.161591,bird
150,bird_aug_24.png,227,0.846290,0.132159,0.176034,4.995953,0.000000,bird
151,bird_aug_23.png,202,0.158564,0.148515,1.332820,21.129645,0.401687,bird


In [10]:
DRONES_DIR_GT = 'Data/Video_V/drones/gt/'
BIRDS_DIR_GT = 'Data/Video_V/birds/gt/'

In [11]:
for filename in os.listdir(DRONES_DIR_GT):
    drones = pd.read_csv(DRONES_DIR_GT+filename, index_col = 0)
    for obj in range(1, len(drones.columns)):
        item = drones.iloc[:,obj].dropna()
        xcenter = []
        ycenter = []
        for j in item:
            j = re.sub('[^\d\.\ ]', '', j)
            j = list(map(float,(j.split())))
            if len(j) == 4:
                xcenter.append(j[0]+j[2]/2)
                ycenter.append(j[1]+j[3]/2)
        img = plot_trajectory(xcenter, ycenter)
        file = filename[:-4] + '_' + str(obj)
        matrix_coocurrence = get_glcm_features(img)
        label = 'drone'
        final_df = final_df.append({'file': file,
                        'frames': drones.shape[0],
                        'turn_angle': turn_angle(xcenter, ycenter),
#                         'x_freq': frequency(xcenter),
                        'y_freq': frequency(ycenter),
                        'curvature': curvature(xcenter, ycenter),
                        'velocity': velocity(np.column_stack((xcenter, ycenter))),
                        'acceleration': acceleration(np.column_stack((xcenter, ycenter))),
                        'label': label}, ignore_index=True)
        glcm_df = pd.concat([glcm_df, pd.DataFrame(matrix_coocurrence.flatten())], axis=1)
        contrast_df = pd.concat([contrast_df, pd.DataFrame(contrast_feature(matrix_coocurrence).flatten())], axis=1)
        dissim_df = pd.concat([dissim_df, pd.DataFrame(dissimilarity_feature(matrix_coocurrence).flatten())], axis=1)
        homogen_df = pd.concat([homogen_df, pd.DataFrame(homogeneity_feature(matrix_coocurrence).flatten())], axis=1)
        energy_df = pd.concat([energy_df, pd.DataFrame(energy_feature(matrix_coocurrence).flatten())], axis=1)
        corr_df = pd.concat([corr_df, pd.DataFrame(correlation_feature(matrix_coocurrence).flatten())], axis=1)
        asm_df = pd.concat([asm_df, pd.DataFrame(asm_feature(matrix_coocurrence).flatten())], axis=1)

/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be 

In [12]:
for filename in os.listdir(BIRDS_DIR_GT):
    birds = pd.read_csv(BIRDS_DIR_GT+filename, index_col = 0)
    for obj in range(1, len(birds.columns)):
        item = birds.iloc[:,obj].dropna()
        xcenter = []
        ycenter = []
        for j in item:
            j = re.sub('[^\d\.\ ]', '', j)
            j = list(map(float,(j.split())))
            if len(j) == 4:
                xcenter.append(j[0]+j[2]/2)
                ycenter.append(j[1]+j[3]/2)
        img = plot_trajectory(xcenter, ycenter)
        file = filename[:-4] + '_' + str(obj)
        matrix_coocurrence = get_glcm_features(img)
        label = 'bird'
        final_df = final_df.append({'file': file,
                        'frames': birds.shape[0],
                        'turn_angle': turn_angle(xcenter, ycenter),
#                         'x_freq': frequency(xcenter),
                        'y_freq': frequency(ycenter),
                        'curvature': curvature(xcenter, ycenter),
                        'velocity': velocity(np.column_stack((xcenter, ycenter))),
                        'acceleration': acceleration(np.column_stack((xcenter, ycenter))),
                        'label': label}, ignore_index=True)
        glcm_df = pd.concat([glcm_df, pd.DataFrame(matrix_coocurrence.flatten())], axis=1)
        contrast_df = pd.concat([contrast_df, pd.DataFrame(contrast_feature(matrix_coocurrence).flatten())], axis=1)
        dissim_df = pd.concat([dissim_df, pd.DataFrame(dissimilarity_feature(matrix_coocurrence).flatten())], axis=1)
        homogen_df = pd.concat([homogen_df, pd.DataFrame(homogeneity_feature(matrix_coocurrence).flatten())], axis=1)
        energy_df = pd.concat([energy_df, pd.DataFrame(energy_feature(matrix_coocurrence).flatten())], axis=1)
        corr_df = pd.concat([corr_df, pd.DataFrame(correlation_feature(matrix_coocurrence).flatten())], axis=1)
        asm_df = pd.concat([asm_df, pd.DataFrame(asm_feature(matrix_coocurrence).flatten())], axis=1)

/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/maksat.kengeskanov/.conda/envs/ml702-env/lib/python3.9/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be 

In [13]:
final_df

,file,frames,turn_angle,y_freq,curvature,velocity,acceleration,label
0,drone_aug_18.png,163,0.648666,0.184049,0.230862,4.248385,0.028669,drone
1,drone_aug_166.png,320,-0.096697,0.093750,0.302611,4.826906,0.094006,drone
2,drone_aug_63.png,119,0.125782,0.252101,0.362415,13.684275,0.248851,drone
3,drone_aug_161.png,130,0.925548,0.230769,0.275335,3.330667,0.001340,drone
4,drone_aug_121.png,254,0.109802,0.118110,0.253963,4.576391,-0.007963,drone
...,...,...,...,...,...,...,...,...
316,V_BIRD_044_LABELS_1,321,0.004212,0.093458,0.003806,1.098303,-0.000474,bird
317,V_BIRD_006_LABELS_1,312,0.241879,0.117188,0.048873,3.525746,0.001283,bird
318,V_BIRD_006_LABELS_2,312,0.111865,0.256410,0.173818,0.443989,-0.004200,bird
319,V_BIRD_010_LABELS_1,317,-0.146575,0.094637,0.130124,7.101296,0.001416,bird


In [14]:
glcm_df = glcm_df.T.reset_index(drop=True)
contrast_df = contrast_df.T.reset_index(drop=True)
dissim_df = dissim_df.T.reset_index(drop=True)
homogen_df = homogen_df.T.reset_index(drop=True)
energy_df = energy_df.T.reset_index(drop=True)
corr_df = corr_df.T.reset_index(drop=True)
asm_df = asm_df.T.reset_index(drop=True)

In [15]:
final_df.to_csv('Data/BasicWithAugmented.csv', index=False)
glcm_df.to_csv('Data/GLCMWithAugmented.csv', index=False)
contrast_df.to_csv('Data/ContrastWithAugmented.csv', index=False)
dissim_df.to_csv('Data/DissimilarityWithAugmented.csv', index=False)
homogen_df.to_csv('Data/HomogeneityWithAugmented.csv', index=False)
energy_df.to_csv('Data/EnergyWithAugmented.csv', index=False)
corr_df.to_csv('Data/CorrelationWithAugmented.csv', index=False)
asm_df.to_csv('Data/ASMWithAugmented.csv', index=False)